In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv("C:/Users/baeza/OneDrive/Desktop/CS 760 Fall 2020/titanic_data.csv").astype(float)

In [2]:
#df

In [3]:
def check_class(pclass):
    if pclass<2:
        return 1.0
    else:
        return 0.0
    
df["Pclass"]=df["Pclass"].apply(check_class)


In [4]:
def check_adult(age):
    if age<=15:
        return 1.0
    else:
        return 0.0
    
df["Age"]=df["Age"].apply(check_adult)

In [5]:
def check_fam(num):
    if num>0:
        return 1
    else:
        return 0
    
df["Siblings/Spouses Aboard"]=df["Siblings/Spouses Aboard"].apply(check_fam)
df["Parents/Children Aboard"]=df["Parents/Children Aboard"].apply(check_fam)


In [6]:
def check_fare(num):
    if num<10:
        return 1.0
    else:
        return 0.0
    
df["Fare"]=df["Fare"].apply(check_fare)
#df

In [7]:
data = df.to_numpy()

In [8]:
from math import log


# takes in data x and computes entropy of column j 
def entropy(x: np.ndarray, j: int ) -> float:
    p=0
    for row in x:
        if row[j] == 1:
            p+=1
    p = p/len(x) 
    term_1 = -p*log(p, 2) if p> 0 else 0 
    term_2 = - (1-p)*log(1-p,2) if (1-p) > 0 else 0  
    return term_1 + term_2

# takes in data x and and column j   
# assumes label is first column of x 
def cond_entropy(x: np.ndarray, j: int) -> float:
    p00, p01, p10, p11 = 0,0,0,0
    y0, y1 = 0,0
    for row in x:
        if row[0] == 0 and row[j] == 0:
            p00+=1
        elif row[0] == 0 and row[j] == 1:
            p01+=1
            y1+=1
        elif row[0] == 1 and row[j] == 0:
            p10+=1
        else:
            p11+=1
            y1+=1
            
    n=len(x)
    y1 = y1/n
    y0 =1-y1
    if (y1 == 0 or y0 == 0):
        return 0
    
    p00, p01, p10, p11 =  p00/n, p01/n, p10/n, p11/n  
    
    term_1 =  -p00*log(p00/y0, 2) if p00 > 0 else 0 
    term_2 =  -p01*log(p01/y0, 2) if p01 > 0 else 0
    term_3 =  -p10*log(p10/y1, 2) if p10 > 0 else 0
    term_4 =  -p11*log(p11/y1, 2) if p11 > 0 else 0

    return term_1 + term_2 + term_3 + term_4 
    
# takes in data x and and column j   
# assumes label is first column of x 
def mut_info(x: np.ndarray, j: int) -> float:
    return entropy(x,j) - cond_entropy(x,j)

    

In [9]:
print(f"Mutual information of Pclass is {mut_info(data,1)}")
print(f"Mutual information of Sex is {mut_info(data,2)}")
print(f"Mutual information of Age is {mut_info(data,3)}")
print(f"Mutual information of Siblings/Spouses is {mut_info(data,4)}")
print(f"Mutual information of Parents/Children is {mut_info(data,5)}")
print(f"Mutual information of Fare is {mut_info(data,6)}")

Mutual information of Pclass is 0.12852990753894167
Mutual information of Sex is 0.2199535166853407
Mutual information of Age is 0.3958215592539193
Mutual information of Siblings/Spouses is 0.023456210552088463
Mutual information of Parents/Children is 0.09012072631532175
Mutual information of Fare is 0.06571482491715952


In [10]:
class Node:

    def __init__(self, value,left=None, right=None, feature=0):
        self.value = value 
        self.feature = feature
        self.left = left
        self.right = right

    
    def __str__(self, level=0):
        ret = ""
        if self is None:
            ret+=" "
        if self.value is not None:
            ret += ( "_\t"*level + str(self.value)+ "\n" ) if self.value is not None else " "
        else:
            return ""
        left, right = self.left, self.right
        
        ret += right.__str__(level+1) if right else ""
        
        ret += left.__str__(level+1) if left else ""
        
        return ret
    

In [11]:

def same_label(S: np.ndarray)-> bool:    
    label = S[0][0]
    if label == None:
        return True
    for s in S:
        if s[0] != label:
            return False
    return True

def major_label(S: np.ndarray)-> float:
    maj = 0
    for s in S:
        if s[0] == 1:
            maj +=1
    return 1.0 if maj >= len(S) - maj else 0.0  


# Implementation of the ID3 algorithm to build decision tree 
# traning set S, feature subset A
def build_tree(S : np.ndarray, A: list)->Node:
    # Stoping criteria
    
    if len(S) == 0:
        return Node(None, None, None,0)
    # if all samples are labeled by 1 (or 0), return a leaf 1 (or 0)
    if same_label(S):
        value = float(S[0][0])
        return Node(value, None, None,0)
        
    
    # if A is empty, return a leaf whose value =
    # majority of labels in S
    if len(A) == 0:
        return Node(major_label(S), None, None,0)
        
    else:
        
        # let j = argmax_{i \in A} mut_info(S,i)
        info = dict()
        for i in A:
            info[i] = mut_info(S,i)
        j = sorted(info.keys(),key=lambda k: info[k])[-1]
        
        # if all samples in S have same label
        if same_label(S):
            value = float(S[0][0])
            if value is None:
                return  Node(value, None, None, 0)
            return Node(S[0][0], None, None, 0)
        
        else:
            #Sm_j = {(x,y) in S : x_j = m } 
            S0_j = S[S[:,j]==0]
            S1_j = S[S[:,j]==1]
            
            A_j = [k for k in A if k != j]
            T_1 = build_tree(S1_j, A_j) if len(S1_j) > 0 else None
            T_2 = build_tree(S0_j, A_j) if len(S0_j) > 0 else None
            
            #This dictionary is only for pretty printing purposes
            categories = dict({1: ("Pclass", 2), 2:("Sex", 1), 3 :("Age", 18),\
                              4:("Sibs/Spse", 1),\
                               5:("Par/Child", 1), 6:("Fare",35),\
                               0: ("WHY are we", "here?")})
            
            #return Node(f" {categories[j][0]} < {categories[j][1]} ", T_2, T_1, feature=j)
            return Node(f" x_{j} < 0.5 ", T_2, T_1, feature=j)
    
    

# Given a decision tree, get its prediction 
def get_DT_prediction(DT: Node, x: np.array)-> float:
    current_node = DT
    x_temp = np.array(x).ravel()
    j = current_node.feature
    while j > 0:
        if x_temp[j] < 0.5:
            if current_node.left is not None:
                current_node = current_node.left
            else:
                break
            j = current_node.feature
            continue
        else:
            if current_node.right is not None:
                current_node = current_node.right
            else:
                break
            j = current_node.feature
            continue
    return current_node.value 


In [12]:
A = [i for i in range(1, len(data[0]))]
DT = build_tree(data, A)
#print("If condition is true, go left \n")
#print(DT)
string = "If condition is true, go \"left\" \n \n"
%store string > example.txt
%store DT.__str__() >> example.txt

Writing 'string' (str) to file 'example.txt'.
Writing 'DT.__str__()' (str) to file 'example.txt'.


In [13]:
from typing import List
from random import randrange

# Builds a random forest using our decision tree implementation
# Has two modes: "80" which uses a random subset of 80% of the data 
# for each tree, and "features" which excludes one feature for each tree
def build_forest(data: np.ndarray, size: int ,mode: str="80") -> List[Node]:
    forest = []
    data_copy = data
    
    for i in range(size):
        # generate random sample of 80% of data 
        if mode=="80":
            for j in range(len(data)//5):
                data_copy = data
                # select a random element and determine its index
                index = randrange(len(data_copy))
                elem = data_copy[index] 
                # delete it so it is not chosen again 
                data_copy = np.delete(data_copy, index, axis=0)
            
            A = [k for k in range(1, len(data_copy[0]))]
            tree = build_tree(data_copy, A) 
            forest.append(tree)
        
        # generate forest by excluding one of every feature
        else:
            data_copy = data
            
            if(size != len(data_copy[0])-1):
                print("Size of forest must equal number of features")
                return
            # delete feature i+1 since feature at index 0 is survival  
            #data_copy = np.delete(data_copy,i+1, axis=1)
            A = [k for k in range(1, len(data_copy[0])) if k != i+1]
            tree = build_tree(data_copy, A) 
            forest.append(tree)
        
    
    return forest

# prints all the trees in the forest 
def print_forest(forest: List[Node]):
    for tree in forest:
        print(f"Tree{ forest.index(tree)+ 1} \n\n")
        print(tree)
        print("\n")
    

# stirng representation of all the trees in the forest 
def forest_to_str(forest: List[Node]):
    ret =""
    for tree in forest:
        ret+= f"Tree{ forest.index(tree)+ 1} \n\n"
        ret+= tree.__str__() + "\n"
    return ret 


#Given a random forest, get its prediction 
def get_forest_prediction(forest: List[Node], x: np.array) -> float:
    results = []
    for tree in forest:
        results.append(get_DT_prediction(tree,x))
    
    return max(set(results),key=results.count)
        
        

In [14]:
from functools import reduce

# split data into k 
def k_fold_split(data: np.ndarray, k: int)-> np.ndarray:
    data_split = []
    data_copy = data
    subset_size = len(data_copy)//k
    
    # save k subsets  
    for i in range(k):
        subset = []
        # add elements to subset.
        while len(subset) < subset_size and len:
            # select a random element and determine its index
            index = randrange(len(data_copy))
            elem = data_copy[index]
            # save the randomly selected line and  
            # delete it so it is not chosen again 
            subset.append(elem)
            data_copy = np.delete(data_copy, index, axis=0)
        # save subset
        data_split.append(np.asarray(subset))
        
    return data_split


#
def k_fold_cv_DT(data: np.ndarray, k: int, mode: str="tree", forest_size: int= 1)-> np.ndarray:
    k_folds = k_fold_split(data, k)
    results = []
    
    
    # detemine training sets and test sets
    for i in range(k):
        cv = np.array([])
        # get all fold indicies and remove one for testing
        train_folds = [l for l in range(k)]
        train_folds.pop(i)
        
        # collect remaining folds into a data set.   
        for j in train_folds:
            # combine remaining folds into training dataset
            # set cv to be the first remaining fold, 
            # then append the rest
            if j == train_folds[0]:
                cv = k_folds[j]
            else:
                cv = np.concatenate((cv, k_folds[j]), axis=0)
       
        # apply models
        if mode == 'tree':    
            A = [i for i in range(1, len(data[0]))]
            DT = build_tree(cv, A) 
            test_predicts = [get_DT_prediction(DT,np.array(row)) for row in k_folds[i]] 
        elif mode == 'forest_80':    
            RF = build_forest(cv, forest_size, mode ="80") 
            test_predicts = [get_forest_prediction(RF,np.array(row)) for row in k_folds[i]] 
        elif mode == 'forest_features':    
            RF = build_forest(cv, forest_size, mode ="features") 
            test_predicts = [get_forest_prediction(RF,np.array(row)) for row in k_folds[i]]
            
        #calculate accuracy
        acc = float(reduce(np.add, list(map(lambda r: 1 if r[0]==r[1][0] else 0, zip(test_predicts, k_folds[i][:])))))
        results.append(acc/len(test_predicts)) 
    
    return sum(results)/len(results)
    
    


In [15]:
k = 10
kcv = k_fold_cv_DT(data,k, mode="tree")
print(f"{k}-fold validation for our decision tree classifier is {kcv}")

10-fold validation for our decision tree classifier is 0.7852272727272728


In [16]:
# our implementation assumes a vector of length 7
# the first value is not used when checking decision tree
x_new = np.array([21, 1, 0, 19, 1, 0, 35]) 
print(f" Decision tree gives {get_DT_prediction(DT,x_new)}")

 Decision tree gives 1.0


In [17]:
RF = build_forest(data, 5, mode="80")
print(f"Random forest predicts {get_forest_prediction(RF, x_new)} in mode 80")
#print_forest(RF)

string2 = "Random forest predicts in mode 80 \n \n"
%store string2 > example2.txt
%store forest_to_str(RF) >> example2.txt

Random forest predicts 1.0 in mode 80
Writing 'string2' (str) to file 'example2.txt'.
Writing 'forest_to_str(RF)' (str) to file 'example2.txt'.


In [18]:
RF2 = build_forest(data, 6, mode="features")
print(f"Random forest predicts {get_forest_prediction(RF2, x_new)} in mode features")
#print_forest(RF2)

string3 = "Random forest predicts in mode features \n \n"
%store string3 > example3.txt
%store forest_to_str(RF2) >> example3.txt

Random forest predicts 1.0 in mode features
Writing 'string3' (str) to file 'example3.txt'.
Writing 'forest_to_str(RF2)' (str) to file 'example3.txt'.


In [19]:
kcv2 = k_fold_cv_DT(data,k, mode="forest_80", forest_size=5)
print(f"{k}-fold validation for our random forest classifier \
which uses 80% random data is {kcv2}")

10-fold validation for our random forest classifier which uses 80% random data is 0.778409090909091


In [20]:
kcv3 = k_fold_cv_DT(data,k, mode="forest_features",forest_size=6)
print(f"{k}-fold validation for our random forest classifier \
which excludes each feature once is {kcv3}")

10-fold validation for our random forest classifier which excludes each feature once is 0.7943181818181818
